In [1]:
# import pandas lib as pd
import pandas as pd
import re
import datetime as dt
import numpy as np
# read 2nd sheet of an excel file
ddid = pd.read_excel(r"C:\Users\jal32\Documents\NSS\Python\Projects\ddid-rasberry-pi\Data\TEIS_Exit_Data.xlsx", sheet_name = 3)

In [2]:
ddid = pd.DataFrame(ddid)

In [3]:
ddid_mvp = ddid[['Child ID', 'DOB', 'Initial IFSP Date', 'Exit Date', 'Days from I-IFSP to Exit', 'Exit Reason', 'Exit Note']]

In [4]:
ddid_mvp_filtered = ddid_mvp[ddid_mvp['Exit Note'].notnull()]

In [5]:
pd.set_option('display.max_colwidth', None)

In [6]:
ddid_mvp_filtered_exit = ddid_mvp_filtered.loc[ddid_mvp_filtered['Exit Note'].str.contains(r"(Exit Date: \d+.\d+.\d+)|(Exit Date:\d+.\d+.\d+)", case=False)]

<ipython-input-6-dc86b58bbfa9>:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  ddid_mvp_filtered_exit = ddid_mvp_filtered.loc[ddid_mvp_filtered['Exit Note'].str.contains(r"(Exit Date: \d+.\d+.\d+)|(Exit Date:\d+.\d+.\d+)", case=False)]


In [7]:
ddid_mvp_keyword_reopened = ddid_mvp_filtered_exit.loc[ddid_mvp_filtered['Exit Note'].str.contains(r"(.eopened: \d+.\d+.\d+)", case=False)]

<ipython-input-7-210d5724d17e>:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  ddid_mvp_keyword_reopened = ddid_mvp_filtered_exit.loc[ddid_mvp_filtered['Exit Note'].str.contains(r"(.eopened: \d+.\d+.\d+)", case=False)]


In [8]:
ddid_clean_col = ddid_mvp_keyword_reopened[['Child ID', 'DOB', 'Initial IFSP Date','Days from I-IFSP to Exit', 'Exit Date', 'Exit Note']]

Verified no duplicates

In [9]:
boolean = ddid["Child ID"].is_unique      # True (credit to @Carsten)
boolean

True

***

In [10]:
ddid_clean_col['exit'] = ddid_clean_col['Exit Note'].str.extract('(Exit Date: \d+.\d+.\d+)')

In [11]:
ddid_clean_col['reopen'] = ddid_clean_col['Exit Note'].str.extract('(Reopened: \d+.\d+.\d+)')

In [12]:
ddid_clean_col['Rev. Exit Date'] = ddid_clean_col['exit'].str.extract('(\d+.\d+.\d+)')

In [13]:
ddid_clean_col['Reopened Date'] = ddid_clean_col['reopen'].str.extract('(\d+.\d+.\d+)')

In [14]:
ddid_clean_col['Reopened Date']= pd.to_datetime(ddid_clean_col['Reopened Date'])

In [15]:
ddid_clean_col['Rev. Exit Date']= pd.to_datetime(ddid_clean_col['Rev. Exit Date'])

In [16]:
ddid_clean_col = ddid_clean_col.drop(columns=['exit', 'reopen'])

In [17]:
ddid_clean_col.dtypes

Child ID                             int64
DOB                         datetime64[ns]
Initial IFSP Date           datetime64[ns]
Days from I-IFSP to Exit             int64
Exit Date                   datetime64[ns]
Exit Note                           object
Rev. Exit Date              datetime64[ns]
Reopened Date               datetime64[ns]
dtype: object

In [18]:
ddid_clean_col['Difference'] = (ddid_clean_col['Reopened Date'] - ddid_clean_col['Rev. Exit Date']).dt.days

In [19]:
ddid_clean_col

,Child ID,DOB,Initial IFSP Date,Days from I-IFSP to Exit,Exit Date,Exit Note,Rev. Exit Date,Reopened Date,Difference
5,473474,2017-11-05,2020-05-06,182,2020-11-04,**LEA county on the Transition page and demographics page do not match due to child moving out of county following the transition meeting. Reopened: 04/06/2020 Exit Date: 11/26/2019 Exit Reason: Ineligible for Part C,2019-11-26,2020-04-06,132
8,469243,2019-07-26,2019-10-02,498,2021-02-11,02-11-2021 – PWN signed and returned for voluntary closure/declined services. Family made aware they can reopen the file if they choose. Reopened: 02/05/2021 Exit Date: 04/07/2020 Exit Reason: 618 - Parent withdraw,2020-04-07,2021-02-05,304
12,436339,2018-04-05,2019-02-13,778,2021-04-01,04/01/2021- File closed business day prior to child's birthday. Parents declined TPC with LEA and plan to continue with private preschool and feeding therapy through insurance. Reopened: 01/11/2019 Exit Date: 06/17/2018 Exit Reason: Ineligible for Part C,2018-06-17,2019-01-11,208
16,474102,2018-05-11,2020-04-09,396,2021-05-10,05/10/2021 TPC held then parents withdrew from testing and thus part B not determined. Reopened: 02/28/2020 Exit Date: 12/05/2019 Exit Reason: Unable to contact,2019-12-05,2020-02-28,85
21,476958,2018-10-17,2020-08-24,274,2021-05-25,05/25/21- parent withdraw Reopened: 08/04/2020 Exit Date: 03/09/2020 Exit Reason: Unable to contact,2020-03-09,2020-08-04,148
...,...,...,...,...,...,...,...,...,...
2223,469136,2017-10-24,2019-10-23,366,2020-10-23,"The family ended TEIS services, and requested to reopen after the transition meeting was already due. Reopened: 07/28/2020 Exit Date: 04/26/2020 Exit Reason: 618 - Parent withdraw",2020-04-26,2020-07-28,93
2227,432033,2017-12-30,2018-04-25,979,2020-12-29,The TPC was late due to LEA getting COVID and then Ryan transferred to foster care and a new placement. Reopened: 02/05/2019 Exit Date: 10/23/2018 Exit Reason: 618 - Parent withdraw,2018-10-23,2019-02-05,105
2232,469330,2018-02-18,2020-05-13,281,2021-02-18,TPC 10/19/2020 Reopened: 03/05/2020 Exit Date: 09/17/2019 Exit Reason: Ineligible for Part C,2019-09-17,2020-03-05,170
2233,428532,2017-07-13,2018-01-26,898,2020-07-12,TPC held and further assessments agreed to; PALS will schedule with family once COVID-19 safety protocals allow. Reopened: 10/25/2018 Exit Date: 05/24/2018 Exit Reason: 618 - Parent withdraw PWN signed on 5/24/18 during May TCM to close file.,2018-05-24,2018-10-25,154


In [20]:
for index, row in ddid_clean_col.iterrows():
    if ((row['Rev. Exit Date'] > row['Initial IFSP Date']) and 
        (row['Rev. Exit Date'] < row['Exit Date']) and
        (row['Reopened Date'] > row['Initial IFSP Date']) and 
        (row['Reopened Date'] < row['Exit Date'])):
        diff_days = (row['Reopened Date'] - row['Rev. Exit Date']).days
        ddid_clean_col.at[index, 'diff_days'] = diff_days
    else:
        ddid_clean_col.at[index, 'diff_days'] = np.nan
        

In [21]:
ddid_clean_col['ED_count'] = ddid_clean_col['Exit Note'].str.count('(Exit Date: \d+.\d+.\d+)')
ddid_clean_col['RD_count'] = ddid_clean_col['Exit Note'].str.count('(Reopened: \d+.\d+.\d+)')

In [22]:
# for index, row in ddid_clean_col.iterrows():
#     if (row['ED_count'] > 1) and (row['RD_count'] > 1):
#         ddid_clean_col.at[index, 'counts'] = 1
#     else:
#         ddid_clean_col.at[index, 'counts'] = 0

In [23]:
# ddid_multiple = ddid_clean_col.loc[ddid_clean_col['counts'] == 1]

In [24]:
exit_dates = (ddid_clean_col['Exit Note'].str.extractall('Exit Date: (\d+.\d+.\d+)')
              .reset_index()
              .rename(columns={'level_0':'index', 
                               0:'e_dates'})
             )
exit_dates
exit_dates['e_dates'] = pd.to_datetime(exit_dates['e_dates'])
exit_dates = exit_dates.drop_duplicates(subset=['index', 'e_dates'])
exit_dates = exit_dates.set_index(['index', 'match']).unstack()
exit_dates.columns = exit_dates.columns.droplevel()
exit_dates.columns = ['exit_' + str(col) for col in exit_dates.columns]
exit_dates

,exit_0,exit_1,exit_2,exit_3,exit_4,exit_5,exit_6
index,,,,,,,
5,2019-11-26,NaT,NaT,NaT,NaT,NaT,NaT
8,2020-04-07,NaT,NaT,NaT,NaT,NaT,NaT
12,2018-06-17,NaT,NaT,NaT,NaT,NaT,NaT
16,2019-12-05,NaT,NaT,NaT,NaT,NaT,NaT
21,2020-03-09,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...
2223,2020-04-26,NaT,NaT,NaT,NaT,NaT,NaT
2227,2018-10-23,NaT,NaT,NaT,NaT,NaT,NaT
2232,2019-09-17,NaT,NaT,NaT,NaT,NaT,NaT


In [25]:
# dummy2 = pd.to_datetime(ddid_multiple['Exit Note'].str.extractall('Reopened: (\d+.\d+.\d+)')[0]).unstack()
# dummy2.columns = ['reopen_' + str(col + 1) for col in dummy2.columns]
reopen_dates = (ddid_clean_col['Exit Note'].str.extractall('Reopened: (\d+.\d+.\d+)')
              .reset_index()
              .rename(columns={'level_0':'index', 
                               0:'r_dates'})
             )
reopen_dates = reopen_dates.drop_duplicates(subset=['index', 'r_dates'])
reopen_dates = reopen_dates.set_index(['index', 'match']).unstack()
reopen_dates.columns = reopen_dates.columns.droplevel()
reopen_dates.columns = ['reopen_' + str(col) for col in reopen_dates.columns]
reopen_dates

,reopen_0,reopen_1,reopen_2,reopen_3,reopen_4,reopen_5
index,,,,,,
5,04/06/2020,NaN,NaN,NaN,NaN,NaN
8,02/05/2021,NaN,NaN,NaN,NaN,NaN
12,01/11/2019,NaN,NaN,NaN,NaN,NaN
16,02/28/2020,NaN,NaN,NaN,NaN,NaN
21,08/04/2020,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2223,07/28/2020,NaN,NaN,NaN,NaN,NaN
2227,02/05/2019,NaN,NaN,NaN,NaN,NaN
2232,03/05/2020,NaN,NaN,NaN,NaN,NaN


In [26]:
all_dates = exit_dates.join(reopen_dates)
cols = all_dates.columns[:]
all_dates[cols] = all_dates[cols].apply(pd.to_datetime, errors='coerce')


In [27]:
pd.set_option('display.max_columns', None)
ddid_full = ddid_clean_col.join(all_dates)
ddid_full

,Child ID,DOB,Initial IFSP Date,Days from I-IFSP to Exit,Exit Date,Exit Note,Rev. Exit Date,Reopened Date,Difference,diff_days,ED_count,RD_count,exit_0,exit_1,exit_2,exit_3,exit_4,exit_5,exit_6,reopen_0,reopen_1,reopen_2,reopen_3,reopen_4,reopen_5
5,473474,2017-11-05,2020-05-06,182,2020-11-04,**LEA county on the Transition page and demographics page do not match due to child moving out of county following the transition meeting. Reopened: 04/06/2020 Exit Date: 11/26/2019 Exit Reason: Ineligible for Part C,2019-11-26,2020-04-06,132,NaN,1,1,2019-11-26,NaT,NaT,NaT,NaT,NaT,NaT,2020-04-06,NaT,NaT,NaT,NaT,NaT
8,469243,2019-07-26,2019-10-02,498,2021-02-11,02-11-2021 – PWN signed and returned for voluntary closure/declined services. Family made aware they can reopen the file if they choose. Reopened: 02/05/2021 Exit Date: 04/07/2020 Exit Reason: 618 - Parent withdraw,2020-04-07,2021-02-05,304,304.0,1,1,2020-04-07,NaT,NaT,NaT,NaT,NaT,NaT,2021-02-05,NaT,NaT,NaT,NaT,NaT
12,436339,2018-04-05,2019-02-13,778,2021-04-01,04/01/2021- File closed business day prior to child's birthday. Parents declined TPC with LEA and plan to continue with private preschool and feeding therapy through insurance. Reopened: 01/11/2019 Exit Date: 06/17/2018 Exit Reason: Ineligible for Part C,2018-06-17,2019-01-11,208,NaN,1,1,2018-06-17,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-11,NaT,NaT,NaT,NaT,NaT
16,474102,2018-05-11,2020-04-09,396,2021-05-10,05/10/2021 TPC held then parents withdrew from testing and thus part B not determined. Reopened: 02/28/2020 Exit Date: 12/05/2019 Exit Reason: Unable to contact,2019-12-05,2020-02-28,85,NaN,1,1,2019-12-05,NaT,NaT,NaT,NaT,NaT,NaT,2020-02-28,NaT,NaT,NaT,NaT,NaT
21,476958,2018-10-17,2020-08-24,274,2021-05-25,05/25/21- parent withdraw Reopened: 08/04/2020 Exit Date: 03/09/2020 Exit Reason: Unable to contact,2020-03-09,2020-08-04,148,NaN,1,1,2020-03-09,NaT,NaT,NaT,NaT,NaT,NaT,2020-08-04,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2223,469136,2017-10-24,2019-10-23,366,2020-10-23,"The family ended TEIS services, and requested to reopen after the transition meeting was already due. Reopened: 07/28/2020 Exit Date: 04/26/2020 Exit Reason: 618 - Parent withdraw",2020-04-26,2020-07-28,93,93.0,1,1,2020-04-26,NaT,NaT,NaT,NaT,NaT,NaT,2020-07-28,NaT,NaT,NaT,NaT,NaT
2227,432033,2017-12-30,2018-04-25,979,2020-12-29,The TPC was late due to LEA getting COVID and then Ryan transferred to foster care and a new placement. Reopened: 02/05/2019 Exit Date: 10/23/2018 Exit Reason: 618 - Parent withdraw,2018-10-23,2019-02-05,105,105.0,1,1,2018-10-23,NaT,NaT,NaT,NaT,NaT,NaT,2019-02-05,NaT,NaT,NaT,NaT,NaT
2232,469330,2018-02-18,2020-05-13,281,2021-02-18,TPC 10/19/2020 Reopened: 03/05/2020 Exit Date: 09/17/2019 Exit Reason: Ineligible for Part C,2019-09-17,2020-03-05,170,NaN,1,1,2019-09-17,NaT,NaT,NaT,NaT,NaT,NaT,2020-03-05,NaT,NaT,NaT,NaT,NaT
2233,428532,2017-07-13,2018-01-26,898,2020-07-12,TPC held and further assessments agreed to; PALS will schedule with family once COVID-19 safety protocals allow. Reopened: 10/25/2018 Exit Date: 05/24/2018 Exit Reason: 618 - Parent withdraw PWN signed on 5/24/18 during May TCM to close file.,2018-05-24,2018-10-25,154,154.0,1,1,2018-05-24,NaT,NaT,NaT,NaT,NaT,NaT,2018-10-25,NaT,NaT,NaT,NaT,NaT


In [28]:
for index, row in ddid_full.iterrows():
    new_diff = 0
    for num in range(0,6):
        if ((row[f'exit_{num}'] != row[f'reopen_{num}']) and
            (row[f'exit_{num}'] != np.nan)  and
            (row[f'reopen_{num}'] != np.nan) and 
            (row[f'exit_{num}'] > row['Initial IFSP Date']) and 
            (row[f'exit_{num}'] < row['Exit Date']) and
            (row[f'reopen_{num}'] > row['Initial IFSP Date']) and 
            (row[f'reopen_{num}'] < row['Exit Date'])):
            mult_diff_days = (row[f'reopen_{num}'] - row[f'exit_{num}']).days
            new_diff += mult_diff_days
            ddid_full.at[index, 'diff_days_new'] = new_diff

        else:
            pass

ddid_full

,Child ID,DOB,Initial IFSP Date,Days from I-IFSP to Exit,Exit Date,Exit Note,Rev. Exit Date,Reopened Date,Difference,diff_days,ED_count,RD_count,exit_0,exit_1,exit_2,exit_3,exit_4,exit_5,exit_6,reopen_0,reopen_1,reopen_2,reopen_3,reopen_4,reopen_5,diff_days_new
5,473474,2017-11-05,2020-05-06,182,2020-11-04,**LEA county on the Transition page and demographics page do not match due to child moving out of county following the transition meeting. Reopened: 04/06/2020 Exit Date: 11/26/2019 Exit Reason: Ineligible for Part C,2019-11-26,2020-04-06,132,NaN,1,1,2019-11-26,NaT,NaT,NaT,NaT,NaT,NaT,2020-04-06,NaT,NaT,NaT,NaT,NaT,NaN
8,469243,2019-07-26,2019-10-02,498,2021-02-11,02-11-2021 – PWN signed and returned for voluntary closure/declined services. Family made aware they can reopen the file if they choose. Reopened: 02/05/2021 Exit Date: 04/07/2020 Exit Reason: 618 - Parent withdraw,2020-04-07,2021-02-05,304,304.0,1,1,2020-04-07,NaT,NaT,NaT,NaT,NaT,NaT,2021-02-05,NaT,NaT,NaT,NaT,NaT,304.0
12,436339,2018-04-05,2019-02-13,778,2021-04-01,04/01/2021- File closed business day prior to child's birthday. Parents declined TPC with LEA and plan to continue with private preschool and feeding therapy through insurance. Reopened: 01/11/2019 Exit Date: 06/17/2018 Exit Reason: Ineligible for Part C,2018-06-17,2019-01-11,208,NaN,1,1,2018-06-17,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-11,NaT,NaT,NaT,NaT,NaT,NaN
16,474102,2018-05-11,2020-04-09,396,2021-05-10,05/10/2021 TPC held then parents withdrew from testing and thus part B not determined. Reopened: 02/28/2020 Exit Date: 12/05/2019 Exit Reason: Unable to contact,2019-12-05,2020-02-28,85,NaN,1,1,2019-12-05,NaT,NaT,NaT,NaT,NaT,NaT,2020-02-28,NaT,NaT,NaT,NaT,NaT,NaN
21,476958,2018-10-17,2020-08-24,274,2021-05-25,05/25/21- parent withdraw Reopened: 08/04/2020 Exit Date: 03/09/2020 Exit Reason: Unable to contact,2020-03-09,2020-08-04,148,NaN,1,1,2020-03-09,NaT,NaT,NaT,NaT,NaT,NaT,2020-08-04,NaT,NaT,NaT,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2223,469136,2017-10-24,2019-10-23,366,2020-10-23,"The family ended TEIS services, and requested to reopen after the transition meeting was already due. Reopened: 07/28/2020 Exit Date: 04/26/2020 Exit Reason: 618 - Parent withdraw",2020-04-26,2020-07-28,93,93.0,1,1,2020-04-26,NaT,NaT,NaT,NaT,NaT,NaT,2020-07-28,NaT,NaT,NaT,NaT,NaT,93.0
2227,432033,2017-12-30,2018-04-25,979,2020-12-29,The TPC was late due to LEA getting COVID and then Ryan transferred to foster care and a new placement. Reopened: 02/05/2019 Exit Date: 10/23/2018 Exit Reason: 618 - Parent withdraw,2018-10-23,2019-02-05,105,105.0,1,1,2018-10-23,NaT,NaT,NaT,NaT,NaT,NaT,2019-02-05,NaT,NaT,NaT,NaT,NaT,105.0
2232,469330,2018-02-18,2020-05-13,281,2021-02-18,TPC 10/19/2020 Reopened: 03/05/2020 Exit Date: 09/17/2019 Exit Reason: Ineligible for Part C,2019-09-17,2020-03-05,170,NaN,1,1,2019-09-17,NaT,NaT,NaT,NaT,NaT,NaT,2020-03-05,NaT,NaT,NaT,NaT,NaT,NaN
2233,428532,2017-07-13,2018-01-26,898,2020-07-12,TPC held and further assessments agreed to; PALS will schedule with family once COVID-19 safety protocals allow. Reopened: 10/25/2018 Exit Date: 05/24/2018 Exit Reason: 618 - Parent withdraw PWN signed on 5/24/18 during May TCM to close file.,2018-05-24,2018-10-25,154,154.0,1,1,2018-05-24,NaT,NaT,NaT,NaT,NaT,NaT,2018-10-25,NaT,NaT,NaT,NaT,NaT,154.0


In [29]:
for index, row in ddid_full.iterrows():
    age_days = (row[f'Exit Date'] - row[f'DOB']).days
    print(age_days)
    if age_days >= 1095:
        ddid_full.at[index, 'Aged Out'] = True
    else:
        ddid_full.at[index, 'Aged Out'] = False

1095
566
1092
1095
951
987
1095
437
1045
1093
1095
811
552
770
1095
1095
1093
1093
1094
631
992
898
1095
1095
1095
1095
827
1095
1095
1095
994
1095
746
1057
635
901
901
1095
560
800
801
844
1095
902
912
912
775
1096
921
1044
1095
1095
1095
450
646
879
862
866
1096
923
850
1095
634
1095
1095
1095
1095
1095
1095
1095
1095
1095
1095
1095
1095
1095
473
1095
1095
780
1095
1095
1095
1095
370
964
1095
1058
1095
1095
1095
1095
958
1095
755
396
612
1095
828
1095
1096
710
993
973
1095
1095
645
458
1095
1008
970
1095
1095
1095
1093
1095
1095
769
1095
1095
1095
1095
669
843
388
1095
1095
1095
805
1094
841
580
1057
1050
750
1095
1095
1095
1093
763
581
1001
1095
741
1030
981
1095
641
432
1095
1095
1095
1095
1095
1095
1095
994
1093
1095
1095
1095
1095
1095
887
1095
1095
1096
1094
1095
1094
1095
836
1095
1095
1095
1095
1095
931
1095
1078
1095
509
598
1095
602
944
1095
534
1043
1095
1009
916
1095
634
497
946
663
834
1095
938
1095
1095
1095
1095
1095
1095
568
1095
1095
716
1095
1095
1095
606
1096
1095
6

In [30]:
ddid_full

,Child ID,DOB,Initial IFSP Date,Days from I-IFSP to Exit,Exit Date,Exit Note,Rev. Exit Date,Reopened Date,Difference,diff_days,ED_count,RD_count,exit_0,exit_1,exit_2,exit_3,exit_4,exit_5,exit_6,reopen_0,reopen_1,reopen_2,reopen_3,reopen_4,reopen_5,diff_days_new,Aged Out
5,473474,2017-11-05,2020-05-06,182,2020-11-04,**LEA county on the Transition page and demographics page do not match due to child moving out of county following the transition meeting. Reopened: 04/06/2020 Exit Date: 11/26/2019 Exit Reason: Ineligible for Part C,2019-11-26,2020-04-06,132,NaN,1,1,2019-11-26,NaT,NaT,NaT,NaT,NaT,NaT,2020-04-06,NaT,NaT,NaT,NaT,NaT,NaN,True
8,469243,2019-07-26,2019-10-02,498,2021-02-11,02-11-2021 – PWN signed and returned for voluntary closure/declined services. Family made aware they can reopen the file if they choose. Reopened: 02/05/2021 Exit Date: 04/07/2020 Exit Reason: 618 - Parent withdraw,2020-04-07,2021-02-05,304,304.0,1,1,2020-04-07,NaT,NaT,NaT,NaT,NaT,NaT,2021-02-05,NaT,NaT,NaT,NaT,NaT,304.0,False
12,436339,2018-04-05,2019-02-13,778,2021-04-01,04/01/2021- File closed business day prior to child's birthday. Parents declined TPC with LEA and plan to continue with private preschool and feeding therapy through insurance. Reopened: 01/11/2019 Exit Date: 06/17/2018 Exit Reason: Ineligible for Part C,2018-06-17,2019-01-11,208,NaN,1,1,2018-06-17,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-11,NaT,NaT,NaT,NaT,NaT,NaN,False
16,474102,2018-05-11,2020-04-09,396,2021-05-10,05/10/2021 TPC held then parents withdrew from testing and thus part B not determined. Reopened: 02/28/2020 Exit Date: 12/05/2019 Exit Reason: Unable to contact,2019-12-05,2020-02-28,85,NaN,1,1,2019-12-05,NaT,NaT,NaT,NaT,NaT,NaT,2020-02-28,NaT,NaT,NaT,NaT,NaT,NaN,True
21,476958,2018-10-17,2020-08-24,274,2021-05-25,05/25/21- parent withdraw Reopened: 08/04/2020 Exit Date: 03/09/2020 Exit Reason: Unable to contact,2020-03-09,2020-08-04,148,NaN,1,1,2020-03-09,NaT,NaT,NaT,NaT,NaT,NaT,2020-08-04,NaT,NaT,NaT,NaT,NaT,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2223,469136,2017-10-24,2019-10-23,366,2020-10-23,"The family ended TEIS services, and requested to reopen after the transition meeting was already due. Reopened: 07/28/2020 Exit Date: 04/26/2020 Exit Reason: 618 - Parent withdraw",2020-04-26,2020-07-28,93,93.0,1,1,2020-04-26,NaT,NaT,NaT,NaT,NaT,NaT,2020-07-28,NaT,NaT,NaT,NaT,NaT,93.0,True
2227,432033,2017-12-30,2018-04-25,979,2020-12-29,The TPC was late due to LEA getting COVID and then Ryan transferred to foster care and a new placement. Reopened: 02/05/2019 Exit Date: 10/23/2018 Exit Reason: 618 - Parent withdraw,2018-10-23,2019-02-05,105,105.0,1,1,2018-10-23,NaT,NaT,NaT,NaT,NaT,NaT,2019-02-05,NaT,NaT,NaT,NaT,NaT,105.0,True
2232,469330,2018-02-18,2020-05-13,281,2021-02-18,TPC 10/19/2020 Reopened: 03/05/2020 Exit Date: 09/17/2019 Exit Reason: Ineligible for Part C,2019-09-17,2020-03-05,170,NaN,1,1,2019-09-17,NaT,NaT,NaT,NaT,NaT,NaT,2020-03-05,NaT,NaT,NaT,NaT,NaT,NaN,True
2233,428532,2017-07-13,2018-01-26,898,2020-07-12,TPC held and further assessments agreed to; PALS will schedule with family once COVID-19 safety protocals allow. Reopened: 10/25/2018 Exit Date: 05/24/2018 Exit Reason: 618 - Parent withdraw PWN signed on 5/24/18 during May TCM to close file.,2018-05-24,2018-10-25,154,154.0,1,1,2018-05-24,NaT,NaT,NaT,NaT,NaT,NaT,2018-10-25,NaT,NaT,NaT,NaT,NaT,154.0,True


In [31]:
ddid_full.loc[ddid_full['Child ID'] == 433605]

,Child ID,DOB,Initial IFSP Date,Days from I-IFSP to Exit,Exit Date,Exit Note,Rev. Exit Date,Reopened Date,Difference,diff_days,ED_count,RD_count,exit_0,exit_1,exit_2,exit_3,exit_4,exit_5,exit_6,reopen_0,reopen_1,reopen_2,reopen_3,reopen_4,reopen_5,diff_days_new,Aged Out
606,433605,2018-01-26,2018-05-23,940,2020-12-18,"Reopened: 01/05/2021 Exit Date: 12/18/2020 Exit Reason: 618 - Attempts to contact unsuccessful Exit Date: 12/18/2020 Exit Reason: Loss of Contact Reopened: 11/02/2020 Exit Date: 07/27/2020 Exit Reason: 618 - Attempts to contact unsuccessful Exit Date: 07/27/2020 SC allowed ten days following sending closing letter, PWN and Right brochure via certified mail, then closed file due to loss of contact with family. Reopened: 07/09/2020 Exit Date: 05/20/2019 Exit Reason: 618 - Parent withdraw Exit Date: 5/20/2019 Paren withdraw Reopened: 12/17/2018 Exit Date: 11/10/2018 Exit Reason: 618 - Parent withdraw",2020-12-18,2021-01-05,18,NaN,7,4,2020-12-18,NaT,2020-07-27,NaT,2019-05-20,NaT,2018-11-10,2021-01-05,2020-11-02,2020-07-09,2018-12-17,NaT,NaT,-18.0,False


In [32]:
ddid_full.loc[ddid_full.diff_days_new < 0]

,Child ID,DOB,Initial IFSP Date,Days from I-IFSP to Exit,Exit Date,Exit Note,Rev. Exit Date,Reopened Date,Difference,diff_days,ED_count,RD_count,exit_0,exit_1,exit_2,exit_3,exit_4,exit_5,exit_6,reopen_0,reopen_1,reopen_2,reopen_3,reopen_4,reopen_5,diff_days_new,Aged Out
606,433605,2018-01-26,2018-05-23,940,2020-12-18,"Reopened: 01/05/2021 Exit Date: 12/18/2020 Exit Reason: 618 - Attempts to contact unsuccessful Exit Date: 12/18/2020 Exit Reason: Loss of Contact Reopened: 11/02/2020 Exit Date: 07/27/2020 Exit Reason: 618 - Attempts to contact unsuccessful Exit Date: 07/27/2020 SC allowed ten days following sending closing letter, PWN and Right brochure via certified mail, then closed file due to loss of contact with family. Reopened: 07/09/2020 Exit Date: 05/20/2019 Exit Reason: 618 - Parent withdraw Exit Date: 5/20/2019 Paren withdraw Reopened: 12/17/2018 Exit Date: 11/10/2018 Exit Reason: 618 - Parent withdraw",2020-12-18,2021-01-05,18,NaN,7,4,2020-12-18,NaT,2020-07-27,NaT,2019-05-20,NaT,2018-11-10,2021-01-05,2020-11-02,2020-07-09,2018-12-17,NaT,NaT,-18.0,False
771,453888,2018-05-28,2019-03-01,819,2021-05-28,Reopened: 02/02/2021 Exit Date: 02/03/2021 Exit Reason: 618 - Part B eligibility not determined,2021-02-03,2021-02-02,-1,-1.0,1,1,2021-02-03,NaT,NaT,NaT,NaT,NaT,NaT,2021-02-02,NaT,NaT,NaT,NaT,NaT,-1.0,True
924,462734,2019-03-12,2019-06-26,638,2021-03-25,Reopened: 03/04/2021 Exit Date: 03/23/2021 Exit Reason: 618 - Parent withdraw Exit Date: 08/15/2019 Exit Reason: 618 - Attempts to contact unsuccessful 03/25/2021: IFSP Goal Met - Child become no longer eligible for part C.,2021-03-23,2021-03-04,-19,-19.0,2,1,2021-03-23,2019-08-15,NaT,NaT,NaT,NaT,NaT,2021-03-04,NaT,NaT,NaT,NaT,NaT,-19.0,False
1952,478049,2018-07-26,2020-04-08,324,2021-02-26,Reopened: 10/29/2020 Exit Date: 10/30/2020 Exit Reason: 618 - Attempts to contact unsuccessful,2020-10-30,2020-10-29,-1,-1.0,1,1,2020-10-30,NaT,NaT,NaT,NaT,NaT,NaT,2020-10-29,NaT,NaT,NaT,NaT,NaT,-1.0,False


In [33]:
ddid_complete = pd.merge(ddid, ddid_full, how = 'left')
ddid_complete.head(25)

,POE,Child ID,DOB,Notification Date,Initial Eligibility Date,Initial IFSP Date,Latest IFSP Date,Exit Reason,Exit Date,Days from I-IFSP to Exit,Exit Note,Rev. Exit Date,Reopened Date,Difference,diff_days,ED_count,RD_count,exit_0,exit_1,exit_2,exit_3,exit_4,exit_5,exit_6,reopen_0,reopen_1,reopen_2,reopen_3,reopen_4,reopen_5,diff_days_new,Aged Out
0,UC,471738,2019-02-28,2019-10-17,2019-10-29,2019-10-31,2020-05-12,618 - Parent withdraw,2020-09-21,326,* No Exit ECO available due to COVID-19,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN
1,FT,449030,2018-07-22,2018-11-07,2018-12-03,2018-12-18,2021-03-04,618 - Moved out of state,2021-03-06,809,"***family moving out of state March 2021, and will complete the Part B eligibility testing in their new state; SC provided EI program information for Missouri to family***",NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN
2,FT,463311,2017-07-09,2019-05-24,2019-06-19,2019-07-02,2020-06-22,618 - Part B eligibility not determined,2020-07-08,372,"**Completed the transition planning conference via 3 way call with family, SC, and LEA due to school closure and the impact of the COVID-19 pandemic. An assessment is TBD. SPED Prek teacher will follow up with family to schedule once school resumes.**",NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN
3,FT,463313,2017-07-09,2019-05-24,2019-06-19,2019-07-02,2020-06-22,618 - Part B eligibility not determined,2020-07-08,372,"**Completed the transition planning conference via 3 way call with family, SC, and LEA due to school closure and the impact of the COVID-19 pandemic. An assessment is TBD. SPED Prek teacher will follow up with family to schedule once school resumes.**",NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN
4,FT,469194,2017-08-31,2019-09-04,2019-10-02,2019-10-11,2020-05-26,618 - Part B eligibility not determined,2020-08-30,324,**Due to the impact of the COVID-19 pandemic and school closure an assessment will be scheduled once school resumes. LEA will contact family to schedule.,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN
5,UC,473474,2017-11-05,2020-04-06,2020-04-24,2020-05-06,2020-06-23,618 - Part B eligibility not determined,2020-11-04,182,**LEA county on the Transition page and demographics page do not match due to child moving out of county following the transition meeting. Reopened: 04/06/2020 Exit Date: 11/26/2019 Exit Reason: Ineligible for Part C,2019-11-26,2020-04-06,132.0,NaN,1.0,1.0,2019-11-26,NaT,NaT,NaT,NaT,NaT,NaT,2020-04-06,NaT,NaT,NaT,NaT,NaT,NaN,True
6,NW,460887,2017-07-19,2019-04-17,2019-05-22,2019-05-29,2020-05-20,618 - Not eligible for Part B – Exit with no referrals,2020-07-18,416,*Continuation of Transition Meeting Conference note of 11-5-19 -- Non LEA service options in the community such as private preschool and daycare were discussed. Speech therapy paid by TennCare can continue after age three. Transition brochure was given. LEA options for a three year old were explained. LEA Process for determining eligibility was explained. Testing will be completed and if eligible Individualized Education Plan (IEP) will be written prior to child's third birthday. Plans are for IP to be written prior to the end of this school year. SC will visit child at Early Head Start this morning for monthly Targeted Case Management (TCM) visit.,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN
7,SE,480267,2018-01-30,2020-03-20,2020-04-28,2020-04-30,2020-10-06,618 - Parent withdraw,2021-01-19,264,01/19/2021- File closed. Parent withdraw due to progress.,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN
8,GN,469243,2019-07-26,2019-09-04,2019-10-01,2019-10-02,2020-03-19,618 - Parent withdraw,2021-02-11,498,02-11-2021 – PWN signed and returned for voluntary closure/declined services. Family made aware th

In [34]:
ddid_complete.pivot_table()

ValueError: No group keys passed!

In [ ]:
eco = pd.read_excel(r"C:\Users\jal32\Documents\NSS\Python\Projects\ddid-rasberry-pi\Data\TEIS_Exit_Data.xlsx", sheet_name = 2)

In [ ]:
ddid.groupby('District')['diff_days_new'].sum()

In [35]:
ddid_complete['new_IFSP_to_exit'] = ddid_complete['Days from I-IFSP to Exit'] - ddid_complete['diff_days_new']

In [39]:
ddid_complete

,POE,Child ID,DOB,Notification Date,Initial Eligibility Date,Initial IFSP Date,Latest IFSP Date,Exit Reason,Exit Date,Days from I-IFSP to Exit,Exit Note,Rev. Exit Date,Reopened Date,Difference,diff_days,ED_count,RD_count,exit_0,exit_1,exit_2,exit_3,exit_4,exit_5,exit_6,reopen_0,reopen_1,reopen_2,reopen_3,reopen_4,reopen_5,diff_days_new,Aged Out,new_IFSP_to_exit
0,UC,471738,2019-02-28,2019-10-17,2019-10-29,2019-10-31,2020-05-12,618 - Parent withdraw,2020-09-21,326,* No Exit ECO available due to COVID-19,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
1,FT,449030,2018-07-22,2018-11-07,2018-12-03,2018-12-18,2021-03-04,618 - Moved out of state,2021-03-06,809,"***family moving out of state March 2021, and will complete the Part B eligibility testing in their new state; SC provided EI program information for Missouri to family***",NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
2,FT,463311,2017-07-09,2019-05-24,2019-06-19,2019-07-02,2020-06-22,618 - Part B eligibility not determined,2020-07-08,372,"**Completed the transition planning conference via 3 way call with family, SC, and LEA due to school closure and the impact of the COVID-19 pandemic. An assessment is TBD. SPED Prek teacher will follow up with family to schedule once school resumes.**",NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
3,FT,463313,2017-07-09,2019-05-24,2019-06-19,2019-07-02,2020-06-22,618 - Part B eligibility not determined,2020-07-08,372,"**Completed the transition planning conference via 3 way call with family, SC, and LEA due to school closure and the impact of the COVID-19 pandemic. An assessment is TBD. SPED Prek teacher will follow up with family to schedule once school resumes.**",NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
4,FT,469194,2017-08-31,2019-09-04,2019-10-02,2019-10-11,2020-05-26,618 - Part B eligibility not determined,2020-08-30,324,**Due to the impact of the COVID-19 pandemic and school closure an assessment will be scheduled once school resumes. LEA will contact family to schedule.,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5577,MD,491609,2018-12-22,2020-10-20,2020-10-30,2020-11-09,2021-03-05,618 - Attempts to contact unsuccessful,2021-05-17,189,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
5578,GN,491743,2018-06-05,2020-10-22,2020-11-05,2020-11-09,2021-05-04,618 - Part B eligibility not determined,2021-06-04,207,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
5579,ET,491879,2019-04-24,2020-10-26,2020-11-02,2020-11-04,2021-04-15,618 - Parent withdraw,2021-05-06,183,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN
5580,SC,492267,2018-06-01,2020-10-30,2020-11-17,2020-11-24,2021-04-15,618 - Part B eligibility not determined,2021-05-31,188,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,NaN,NaN


In [40]:
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
ddid_complete.to_csv('ddid_complete.zip', index=False,
          compression=compression_opts)

In [42]:
ddid_complete['Exit Reason'].value_counts()

618 - Part B eligibility not determined                                 1958
618 - Parent withdraw                                                   1243
618 - Part B eligible                                                   1173
618 - Attempts to contact unsuccessful                                   566
618 - Not eligible for Part B – Exit with referral to other programs     245
618 - Moved out of state                                                 168
618 - Not eligible for Part B – Exit with no referrals                   162
618 - Completion of IFSP/ no longer requires services                     57
618 - Deceased                                                            10
Name: Exit Reason, dtype: int64

In [44]:
for index, row in ddid_complete.iterrows():
    age_days = (row[f'Exit Date'] - row[f'DOB']).days
    if age_days >= 1095:
        ddid_complete.at[index, 'Aged Out'] = True
    else:
        ddid_complete.at[index, 'Aged Out'] = False

In [45]:
ddid_complete

,POE,Child ID,DOB,Notification Date,Initial Eligibility Date,Initial IFSP Date,Latest IFSP Date,Exit Reason,Exit Date,Days from I-IFSP to Exit,Exit Note,Rev. Exit Date,Reopened Date,Difference,diff_days,ED_count,RD_count,exit_0,exit_1,exit_2,exit_3,exit_4,exit_5,exit_6,reopen_0,reopen_1,reopen_2,reopen_3,reopen_4,reopen_5,diff_days_new,Aged Out,new_IFSP_to_exit
0,UC,471738,2019-02-28,2019-10-17,2019-10-29,2019-10-31,2020-05-12,618 - Parent withdraw,2020-09-21,326,* No Exit ECO available due to COVID-19,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,False,NaN
1,FT,449030,2018-07-22,2018-11-07,2018-12-03,2018-12-18,2021-03-04,618 - Moved out of state,2021-03-06,809,"***family moving out of state March 2021, and will complete the Part B eligibility testing in their new state; SC provided EI program information for Missouri to family***",NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,False,NaN
2,FT,463311,2017-07-09,2019-05-24,2019-06-19,2019-07-02,2020-06-22,618 - Part B eligibility not determined,2020-07-08,372,"**Completed the transition planning conference via 3 way call with family, SC, and LEA due to school closure and the impact of the COVID-19 pandemic. An assessment is TBD. SPED Prek teacher will follow up with family to schedule once school resumes.**",NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,True,NaN
3,FT,463313,2017-07-09,2019-05-24,2019-06-19,2019-07-02,2020-06-22,618 - Part B eligibility not determined,2020-07-08,372,"**Completed the transition planning conference via 3 way call with family, SC, and LEA due to school closure and the impact of the COVID-19 pandemic. An assessment is TBD. SPED Prek teacher will follow up with family to schedule once school resumes.**",NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,True,NaN
4,FT,469194,2017-08-31,2019-09-04,2019-10-02,2019-10-11,2020-05-26,618 - Part B eligibility not determined,2020-08-30,324,**Due to the impact of the COVID-19 pandemic and school closure an assessment will be scheduled once school resumes. LEA will contact family to schedule.,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5577,MD,491609,2018-12-22,2020-10-20,2020-10-30,2020-11-09,2021-03-05,618 - Attempts to contact unsuccessful,2021-05-17,189,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,False,NaN
5578,GN,491743,2018-06-05,2020-10-22,2020-11-05,2020-11-09,2021-05-04,618 - Part B eligibility not determined,2021-06-04,207,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,True,NaN
5579,ET,491879,2019-04-24,2020-10-26,2020-11-02,2020-11-04,2021-04-15,618 - Parent withdraw,2021-05-06,183,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,False,NaN
5580,SC,492267,2018-06-01,2020-10-30,2020-11-17,2020-11-24,2021-04-15,618 - Part B eligibility not determined,2021-05-31,188,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaN,True,NaN


In [47]:
ddid_complete['Aged Out'].value_counts()

True     3261
False    2321
Name: Aged Out, dtype: int64